In [1]:
import pandas as pd
import requests
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Float, PrimaryKeyConstraint, func,inspect,text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
import sqlite3
from sqlalchemy.ext.automap import automap_base
import config
from config import geoapify_key

In [2]:
url="https://api.census.gov/data/2019/pep/charagegroups?"

In [3]:
all_population="000"
age_65_up="026"
query_url=url+ "get=NAME,POP&for=county:*&in=state:12&AGEGROUP="
all_pop_query=query_url+all_population
age_65_up_query=query_url+age_65_up
results_cols=["County, State","Population","Age Group","State Code","County Code"]

In [4]:
all_pop_results=requests.get(all_pop_query).json()
age_65_up_results=requests.get(age_65_up_query).json()

In [5]:
age_65_up_df=pd.DataFrame(age_65_up_results,columns=results_cols)
age_65_up_df.columns=age_65_up_df.iloc[0]
age_65_up_df=age_65_up_df.tail(-1)

In [6]:
all_pop_df=pd.DataFrame(all_pop_results,columns=results_cols)
all_pop_df.columns=all_pop_df.iloc[0]
all_pop_df=all_pop_df.tail(-1)

In [7]:
pop_percent_df=pd.merge(all_pop_df,age_65_up_df,on='county',how='outer')

In [8]:
pop_percent_df[['County','State']]=pop_percent_df['NAME_x'].str.split(', ',expand=True)
pop_percent_df.drop(columns=['NAME_x','NAME_y','AGEGROUP_x','AGEGROUP_y','state_x','state_y','State'],inplace=True)

In [9]:
pop_percent_df=pop_percent_df.rename(columns={'county':'County_ID','POP_x':'Total_Population','POP_y':'Elderly_Population','state':'State_ID'})
data_types={'Total_Population':'int','County_ID':'int','Elderly_Population':'int','County':'str'}
pop_percent_df=pop_percent_df.astype(data_types)

In [10]:
pop_percent_df['Percent_Elderly']=round((pop_percent_df['Elderly_Population']/pop_percent_df['Total_Population'])*100,2)
# pop_percent_df['Percent_Elderly'] = pop_percent_df['Percent_Elderly'].map('{:.2f}'.format)
pop_percent_df['County'] = pop_percent_df['County'].str.replace(' County', '')

In [19]:
hospital_url = "https://api.geoapify.com/v2/places?categories=healthcare.hospital&filter=place:512bfac038c5a254c059f90f01cee96b3c40f00101f9010279020000000000920307466c6f72696461&limit=300&apiKey="+geoapify_key    
hospital_response = requests.get(hospital_url).json()
hospital_information = []
features = hospital_response["features"]

for feature in features:
    properties = feature['properties']
    hospital_name = properties.get('name')
    hospital_county = properties.get('county')
    hospital_city = properties.get('city')
    hospital_lat = properties.get('lat')  
    hospital_lon = properties.get('lon')   

    # Check if both keys are defined before appending to hospital_information
    if hospital_name is not None and hospital_county is not None:
        hospital_information.append({'Name': hospital_name, 'City': hospital_city,'County':hospital_county,'Latitude':hospital_lat,'Longitude':hospital_lon})
hospital_df=pd.DataFrame(hospital_information)
hospital_df['ID'] = range(1, len(hospital_df) + 1)
hospital_df['County'] = hospital_df['County'].str.replace(' County', '')
hospital_df.head()

,Name,City,County,Latitude,Longitude,ID
0,Johns Hopkins All Children's Hospital,Saint Petersburg,Pinellas,27.762944,-82.639751,1
1,Tampa General Hospital,Tampa,Hillsborough,27.937501,-82.458900,2
2,Morton Plant North Bay Hospital,New Port Richey,Pasco,28.254698,-82.714225,3
3,AdventHealth Tampa,Tampa,Hillsborough,28.071491,-82.421897,4
4,Saint Joseph's Hospital,Tampa,Hillsborough,27.982750,-82.491464,5


In [15]:
conn = sqlite3.connect('florida_info_db.sqlite')
pop_percent_df.to_sql('elderly_people', conn, index=False, if_exists='replace', dtype={'County_ID': 'INTEGER PRIMARY KEY'})


67

In [20]:
hospital_df.to_sql('hospitals', conn, index=False, if_exists='replace', dtype={'ID': 'INTEGER PRIMARY KEY'})

85